In [1]:
import bo_methods_lib
from bo_methods_lib.CS2_bo_functions_multi_dim import LHS_Design, test_train_split
from bo_methods_lib.CS2_create_data import create_y_data, calc_y_exp, create_sse_data, calc_muller
from bo_methods_lib.bo_functions_generic import clean_1D_arrays

from scipy.stats import qmc
import torch
import numpy as np
import pandas as pd
import os

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
#LHS Design in python - May switch code to use this instead for conciseness
emulator = True
emulator = False
seed = 9
LHS_Points =200
# LHS_Points =20
d = 8
# d = 2
sampler = qmc.LatinHypercube(d=d, seed = seed)
sample = sampler.random(n=LHS_Points)

#Only going to regress "a and b" parameters
skip_param_types = 1 #(Skiiping A)
l_bounds =  [-2, -2, -10, -2, -2, -2,  5, -2]
u_bounds =  [ 2,  2,   0,  2,  2,  2, 15,  2]

# l_bounds =  [-2, -2, -10, -2]
# u_bounds =  [ 2,  2,   0,  2]

# l_bounds =  [-2, -2]
# u_bounds =  [2, 2 ]
LHS = qmc.scale(sample, l_bounds, u_bounds)

In [11]:
#LHS for experimental data points
exp_LHS_Points = 10 #0 , 10, 15, 20, or 25 for 4, 8, 12, 16, and 24 params
exp_d = 2
exp_sampler = qmc.LatinHypercube(d=exp_d, seed = seed)
exp_sample = exp_sampler.random(n=exp_LHS_Points)

exp_l_bounds =  [-1.5, -0.5]
exp_u_bounds =  [1 , 2]
exp_LHS = qmc.scale(exp_sample, exp_l_bounds, exp_u_bounds)
print(exp_LHS)

[[-0.2175623   0.9282957 ]
 [-1.15078704 -0.44438352]
 [-1.42901866  0.27115497]
 [-0.96509841  1.77044059]
 [ 0.74335307  0.640688  ]
 [ 0.87876392  1.73371145]
 [-0.50140651  0.04234464]
 [ 0.00417444  1.05384754]
 [ 0.42109913 -0.17633343]
 [-0.32479527  1.31481288]]


In [12]:
#Creates Experimental data and saves to a csv
# Constants = np.array([-200,-100,-170,15,   #A
#                       -1,-1,-6.5,0.7,    #a
#                       0,0,11,0.6,        #b
#                       -10,-10,-6.5,0.7,  #c
#                       1,0,-0.5,-1,      #x0
#                       0,0.5,1.5,1])     #yo

Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])

# Theta_True = np.array([-2,2])
# Xexp = np.array([-2,-1,0,1,2])

Xexp = np.array([[-0.558,-0.050,0.623,-0.82, 0.22],
                 [1.442,0.467,0.028,0.62,0.30]])

Xexp = np.vstack((exp_LHS, Xexp.T))
print(Xexp[0])
print(Xexp.shape)

n = Xexp.shape[0]
print(n)
noise_std = 0.01
Yexp = calc_y_exp(Constants, Xexp, noise_std)
print(Yexp)

[-0.2175623  0.9282957]
(15, 2)
15
[   5.05474681   74.53890398   12.24428821   16.62324988   44.33836208
  591.55992898   -4.72307241   26.04219455  -46.1040296   -52.96709878
 -146.69947812  -80.76773816 -108.16663897  -40.67072465  -72.22508357]


In [13]:
df = pd.DataFrame({'X1exp': Xexp[:, 0], 'X2exp': Xexp[:, 1], 'Yexp':Yexp})
# df = pd.DataFrame({'Xexp': Xexp, 'Yexp':Yexp})
print(df)

# path = 'Input_CSVs/Exp_Data/d='+str(exp_d)
# if not os.path.exists(path):
#     os.makedirs(path)
# df.to_csv(os.path.join(path,'n='+str(n)+'.csv'))

       X1exp     X2exp        Yexp
0  -0.217562  0.928296    5.054747
1  -1.150787 -0.444384   74.538904
2  -1.429019  0.271155   12.244288
3  -0.965098  1.770441   16.623250
4   0.743353  0.640688   44.338362
5   0.878764  1.733711  591.559929
6  -0.501407  0.042345   -4.723072
7   0.004174  1.053848   26.042195
8   0.421099 -0.176333  -46.104030
9  -0.324795  1.314813  -52.967099
10 -0.558000  1.442000 -146.699478
11 -0.050000  0.467000  -80.767738
12  0.623000  0.028000 -108.166639
13 -0.820000  0.620000  -40.670725
14  0.220000  0.300000  -72.225084


In [14]:
#Array for Standard Approach
all_p_standard = pd.DataFrame(LHS, columns = ["a1", "a2", "a3", "a4", "b1", "b2", "b3", "b4"])
# all_p_standard = pd.DataFrame(LHS, columns = ["a1", "a2", "a3", "a4"])
# all_p_standard = pd.DataFrame(LHS, columns = ["theta1", "theta2"])

print(Xexp,Xexp.shape)
#Array for Emulator approach
df = all_p_standard.iloc[np.repeat(np.arange(len(all_p_standard)), n)]
df = df.reset_index(drop = True)
# Xexp_repeat = np.repeat(Xexp, LHS_Points, axis = 1).reshape(Xexp.shape[1],-1)
# print(Xexp_repeat.reshape(300,2))
Xexp_repeat = np.tile(Xexp, (len(all_p_standard),1))
#Add Xexp 
for i in range(Xexp.shape[1]):
    df['X'+str(i+1)+'exp'] = Xexp_repeat[:,i]    
all_p_emulator = df

print(all_p_emulator)
# print(all_p_standard)


[[-0.2175623   0.9282957 ]
 [-1.15078704 -0.44438352]
 [-1.42901866  0.27115497]
 [-0.96509841  1.77044059]
 [ 0.74335307  0.640688  ]
 [ 0.87876392  1.73371145]
 [-0.50140651  0.04234464]
 [ 0.00417444  1.05384754]
 [ 0.42109913 -0.17633343]
 [-0.32479527  1.31481288]
 [-0.558       1.442     ]
 [-0.05        0.467     ]
 [ 0.623       0.028     ]
 [-0.82        0.62      ]
 [ 0.22        0.3       ]] (15, 2)
            a1        a2        a3        a4        b1        b2         b3  \
0    -0.917405 -0.685736 -1.130157  1.924449 -1.374321 -0.378308  14.306980   
1    -0.917405 -0.685736 -1.130157  1.924449 -1.374321 -0.378308  14.306980   
2    -0.917405 -0.685736 -1.130157  1.924449 -1.374321 -0.378308  14.306980   
3    -0.917405 -0.685736 -1.130157  1.924449 -1.374321 -0.378308  14.306980   
4    -0.917405 -0.685736 -1.130157  1.924449 -1.374321 -0.378308  14.306980   
...        ...       ...       ...       ...       ...       ...        ...   
2995  0.361281 -1.528891 -2.28747

In [15]:
## Create Training Data and Testing Data
#Set noise parameters and load csv file to generate training data
# all_p = LHS_Design("LHS_Toy_3_Input.csv")

#Create y or SSE data
#Save data
if emulator == False:
    all_2_data = all_p_standard.copy()
    all_2_in_y = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "obj", skip_param_types=skip_param_types)
    all_2_data['Yexp'] = all_2_in_y
    
    all_2_data_ln_obj = all_p_standard.copy()
    all_2_in_y_ln_obj = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "LN_obj", skip_param_types=skip_param_types)
    all_2_data_ln_obj['Yexp'] = all_2_in_y_ln_obj
    
    titles_all = np.array(['all_st_data','all_st_ln_obj_data'])
    len_data = str(len(all_2_data))
    all_array = [all_2_data,all_2_data_ln_obj]

else:
    all_3_data = all_p_emulator.copy()
    all_3_in_y = create_y_data(all_p_emulator, Constants, Xexp, skip_param_types=skip_param_types)
    all_3_data['Yexp'] = all_3_in_y
    
    titles_all = np.array(['all_emul_data'])
    len_data = str(len(all_3_data))
    all_array = [all_3_data]
    
# print(all_array)

# Save Train Data/Test Data
for i in range(len(titles_all)):
    path = 'Input_CSVs/Train_Data/d=' + str(d) +"/"+ titles_all[i]+"/"
    if not os.path.exists(path):
        os.makedirs(path)
    all_array[i].to_csv(os.path.join(path,'t='+len_data)+'.csv')